In [7]:
# bronze_to_silver_pollutant.py  ──────────────────────────────────────
import pandas as pd, pathlib, glob, re, html, sys
BRONZE_DEF = pathlib.Path("/workspaces/airoute_mlops/airoute_mlops/data/bronze/defra")
OUTPUT_PATH = pathlib.Path("/workspaces/airoute_mlops/airoute_mlops/data/silver/pollutant")

# ───────── helpers ───────────────────────────────────────────────────
def normalise(col: str) -> str:
    """lower-case, strip HTML <sub>, spaces."""
    return re.sub(r"</?sub>", "", html.unescape(col)).lower().strip()

DESIRED = ["site_id", "date_time", "pm25", "no2", "o3"]
REMAP   = {
    "ozone":                                 "o3",
    "nitrogen dioxide":                      "no2",
    "pm2.5 particulate matter (hourly measured)": "pm25"
}

# ───────── iterate CSVs ──────────────────────────────────────────────
frames = []
for fp in glob.glob(str(BRONZE_DEF / "*.csv")):
    site_id = pathlib.Path(fp).stem.split("_")[0]
    print(f"· Parsing {fp}")

    # 1. read & normalise column names
    df = pd.read_csv(fp, parse_dates=[["date", "time"]])
    df.columns = [normalise(c) for c in df.columns]

    # 2. rename pollutant columns to canonical names
    df = df.rename(columns=REMAP)

    # 3. ensure 'site_id' present
    df["site_id"] = site_id

    # 4. add any missing pollutant columns as NA
    for col in ["pm25", "no2", "o3"]:
        if col not in df.columns:
            df[col] = pd.NA

    # 5. keep exactly desired columns (others are dropped)
    df = df[DESIRED]
        # after df = df[DESIRED]
    df["date_time"] = pd.to_datetime(
        df["date_time"], format="%d-%m-%Y %H:%M", errors="coerce"
    ).dt.tz_localize("UTC")


    # 6. drop rows where ALL three pollutants are NA
    df = df.dropna(subset=["pm25", "no2", "o3"], how="all")

    if df.empty:
        print(f"  ⚠  {site_id}: no pollutant values → skipped")
        continue

    frames.append(df)

# ───────── write parquet ────────────────────────────────────────────
if not frames:
    sys.exit("❌ No valid pollutant rows found in any CSV!")

silver = pd.concat(frames, ignore_index=True)
# OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
# silver.to_parquet(OUTPUT_PATH, index=False)

# print(f"\n✅ Saved {len(silver):,} rows to {OUTPUT_PATH}")

# --- keep your imports / code above unchanged -----------------------

# after you build `silver`   (the DataFrame)
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)         # make the dir once

outfile = OUTPUT_PATH / "pollutant_hourly_2025.parquet"   # ← the file

silver.to_parquet(outfile, index=False)                # write here

print(f"\n✅ Saved {len(silver):,} rows to {outfile}")



· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/BLWD_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/WAL4_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/PEGR_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/SOUT_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/CAM_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/ECCL_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/KIEW_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/GKA8_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/WIG5_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/CNPR_2025.csv


/tmp/ipykernel_2666/4211641774.py:25: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(fp, parse_dates=[["date", "time"]])
/tmp/ipykernel_2666/4211641774.py:25: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(fp, parse_dates=[["date", "time"]])
/tmp/ipykernel_2666/4211641774.py:25: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(fp, parse_dates=[["date", "time"]])
/tmp/ipykernel_2666/4211641774.py:25: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(fp, parse_da

· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/SHLW_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/EAGL_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/STOR_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/WEYB_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/BEL2_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/COPP_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/BRAS_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/CHP_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/MY1_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/HARW_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/BMLD_2025.csv


/tmp/ipykernel_2666/4211641774.py:25: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(fp, parse_dates=[["date", "time"]])
/tmp/ipykernel_2666/4211641774.py:25: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(fp, parse_dates=[["date", "time"]])
/tmp/ipykernel_2666/4211641774.py:25: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(fp, parse_dates=[["date", "time"]])
/tmp/ipykernel_2666/4211641774.py:25: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(fp, parse_da

· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/ESK_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/COBR_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/PLYM_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/NWBV_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/DESB_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/TOFT_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/NTN4_2025.csv


...

· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/SUN2_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/SIB_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/SHDG_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/GHSR_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/SV_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/BRT3_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/YW_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/CA1_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/STKR_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/BR11_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/EDNS_2025.csv


/tmp/ipykernel_2666/4211641774.py:25: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(fp, parse_dates=[["date", "time"]])
/tmp/ipykernel_2666/4211641774.py:25: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(fp, parse_dates=[["date", "time"]])
/tmp/ipykernel_2666/4211641774.py:25: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(fp, parse_dates=[["date", "time"]])
/tmp/ipykernel_2666/4211641774.py:25: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(fp, parse_da

· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/LVPT_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/SEND_2025.csv
· Parsing /workspaces/airoute_mlops/airoute_mlops/data/bronze/defra/LONM_2025.csv


/tmp/ipykernel_2666/4211641774.py:25: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(fp, parse_dates=[["date", "time"]])
/tmp/ipykernel_2666/4211641774.py:25: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(fp, parse_dates=[["date", "time"]])
/tmp/ipykernel_2666/4211641774.py:25: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(fp, parse_dates=[["date", "time"]])
/tmp/ipykernel_2666/4211641774.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the r


✅ Saved 892,080 rows to /workspaces/airoute_mlops/airoute_mlops/data/silver/pollutant/pollutant_hourly_2025.parquet


In [ ]:
data = pd.read_parquet(outfile)  # read back to check
data.info()  # print info about the DataFrame